In [1]:
from pybroker import Strategy, YFinance, highest
from DataSource.qlibdata import qlibDataSource
from database_utils.db_utils import save_to_db, DuckDBManager
import pandas as pd

DB_PATH = "database_utils/duckdb_files"

def exec_fn(ctx):
   # Get the rolling 10 day high.
   high_10d = ctx.indicator('high_10d')
   # Buy on a new 10 day high.
   if not ctx.long_pos() and high_10d[-1] > high_10d[-2]:
      ctx.buy_shares = 100
      # Hold the position for 5 days.
      ctx.hold_bars = 5
      # Set a stop loss of 2%.
      ctx.stop_loss_pct = 2


data_source = qlibDataSource()
symbols = ['SH600809','SZ000001', 'SZ000002']
start_date = '2022-01-01'
end_date = '2023-12-31'
_timeframe = 'day'
_adjust = None

df = data_source._fetch_data(symbols, start_date, end_date, _timeframe, _adjust)
strategy = Strategy(df, start_date=start_date, end_date=end_date)
strategy.add_execution(exec_fn, symbols, indicators=highest('high_10d', 'close', period=10))
# # Run the backtest after 20 days have passed.
result = strategy.backtest(warmup=20)
result.orders
# 将result.orders装为dataframe
# df = pd.DataFrame(result.orders)
# 将数字2转为dataframe
df = pd.DataFrame([2])

save_to_db(DB_PATH,"pybroker.db", "df", df)

[976827:MainThread](2023-09-11 22:31:36,409) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[976827:MainThread](2023-09-11 22:31:36,651) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[976827:MainThread](2023-09-11 22:31:36,652) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/idea/.qlib/qlib_data/invest')}
[976827:MainThread](2023-09-11 22:31:36,892) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[976827:MainThread](2023-09-11 22:31:36,894) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[976827:MainThread](2023-09-11 22:31:36,895) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/idea/.qlib/qlib_data/invest')}


Backtesting: 2022-01-01 00:00:00 to 2023-12-31 00:00:00

Computing indicators...


  0% (0 of 3) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 33% (1 of 3) |########                  | Elapsed Time: 0:00:01 ETA:   0:00:03
100% (3 of 3) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01



Test split: 2022-01-04 00:00:00 to 2023-09-05 00:00:00


  0% (0 of 407) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
 59% (241 of 407) |#############         | Elapsed Time: 0:00:00 ETA:  00:00:00
100% (407 of 407) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:05
┌───────┐
│   0   │
│ int64 │
├───────┤
│     2 │
└───────┘

